In [136]:
import datetime, time, timeit, os, os.path

import numpy as np
import numpy.random as nprd
import numpy.linalg as npalg
import pandas as pd
import scipy.constants as scpcst
import xarray as xr

import plotly.graph_objects as pgo
import plotly.express as px
import plotly.subplots as psp

pd.set_option("display.max_columns", 999)

# Earth Geometry


In [137]:
R = 6371.0088*1E3/scpcst.nautical_mile

In [138]:
def coord_to_vect(lat, lon):
    
    lat, lon = np.broadcast_arrays(lat, lon)
    
    lat = np.deg2rad(lat)
    lon = np.deg2rad(lon)
    
    coslat = np.cos(lat)
    x = coslat*np.cos(lon)
    y = coslat*np.sin(lon)
    z = np.sin(lat)
    
    return x,y,z

In [139]:
def vect_to_coord(x,y,z):
    
    x, y, z = np.broadcast_arrays(x, y, z)
    
    lat = np.rad2deg( np.arctan2(z, np.sqrt(x**2+y**2)) )
    lon = np.rad2deg( np.arctan2(y,x) )
    
    return lat, lon

In [140]:
def distance(lat1, lon1, lat2, lon2):
    
    lat1, lon1, lat2, lon2 = np.broadcast_arrays(lat1, lon1, lat2, lon2)
    
    vect1 = np.stack(coord_to_vect(lat1, lon1), axis=0)
    vect2 = np.stack(coord_to_vect(lat2, lon2), axis=0)
    
    cosine = sum(a*b for a,b in zip(vect1, vect2))
    sine = npalg.norm(np.cross(vect1, vect2, axisa=0, axisb=0, axisc=0), axis=0)
    
    return R*np.arctan2(sine, cosine)

In [141]:
def great_circle(lat1, lon1, lat2, lon2, s):
    """
    s = in nautical miles
    """
    
    alpha = np.array(s)/R
    lat1, lon1, lat2, lon2, alpha = np.broadcast_arrays(lat1, lon1, lat2, lon2, alpha)

    vect1 = np.stack(coord_to_vect(lat1, lon1), axis=0)
    vect2 = np.stack(coord_to_vect(lat2, lon2), axis=0)
    
    tangent = np.cross(np.cross(vect1, vect2, axisa=0, axisb=0, axisc=0), vect1, axisa=0, axisb=0, axisc=0)
    tangent /= npalg.norm(tangent, axis=0)
    
    circle = vect1*np.cos(alpha) + tangent*np.sin(alpha)
    
    return vect_to_coord(*circle)

In [142]:
def great_circle_heading(lat1, lon1, lat2, lon2, s): 
    """
    nautical conventions: clockwise angle from the north!
    """
    
    alpha = np.array(s)/R
    lat1, lon1, lat2, lon2, alpha = np.broadcast_arrays(lat1, lon1, lat2, lon2, alpha)
    
    vect1 = np.stack(coord_to_vect(lat1, lon1), axis=0)
    vect2 = np.stack(coord_to_vect(lat2, lon2), axis=0)
    
    tangent1 = np.cross(np.cross(vect1, vect2, axisa=0, axisb=0, axisc=0), vect1, axisa=0, axisb=0, axisc=0)
    tangent1 /= npalg.norm(tangent1, axis=0)
    
    circle = vect1*np.cos(alpha) + tangent1*np.sin(alpha)
    
    target = np.broadcast_to(tangent1, circle.shape)
    target = np.where(alpha>=np.pi/2, -vect1, target)
    target = np.where(alpha>=np.pi, -tangent1, target)
    target = np.where(alpha>=3*np.pi/2, vect1, target)
    
    tangent = np.cross(np.cross(circle, target, axisa=0, axisb=0, axisc=0), circle, axisa=0, axisb=0, axisc=0)
    tangent /= npalg.norm(tangent, axis=0)
    
    north = np.cross(np.cross(circle, np.array([0,0,1]), axisa=0, axisc=0), circle, axisa=0, axisb=0, axisc=0)
    north /= npalg.norm(north, axis=0)
    
    cosheading = sum(a*b for a,b in zip(tangent, north))
    sinheading = sum(a*b for a,b in zip(circle, np.cross(tangent, north, axisa=0, axisb=0, axisc=0)))
    
    heading = np.arctan2(sinheading, cosheading)*(180/np.pi)
    
    return (*vect_to_coord(*circle), heading)

# Wind

In [328]:
wind_data = xr.open_dataset(r"NetCDF/Suez_Wind_2016_to_2021.nc")
wind_data

<xarray.Dataset>
Dimensions:    (latitude: 13, longitude: 5, time: 46423)
Coordinates:
  * longitude  (longitude) float32 32.0 32.25 32.5 32.75 33.0
  * latitude   (latitude) float32 32.0 31.75 31.5 31.25 ... 29.5 29.25 29.0
  * time       (time) datetime64[ns] 2016-01-01 ... 2021-04-18T06:00:00
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    i10fg      (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2021-04-21 17:30:11 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [329]:
# wind_data = xr.open_dataset(r"NetCDF/Livourne_Italy.nc")
# wind_data

In [330]:
# wind_data = xr.open_dataset(r"NetCDF/Moroni_Comorres.nc")
# wind_data

In [331]:
# wind_data = xr.open_dataset(r"NetCDF/Brazil.nc")
# wind_data

In [332]:
da = wind_data['time']
df = da.to_dataframe()
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
print(df)


                                   time
time                                   
2016-01-01 00:00:00 2016-01-01 00:00:00
2016-01-01 01:00:00 2016-01-01 01:00:00
2016-01-01 02:00:00 2016-01-01 02:00:00
2016-01-01 03:00:00 2016-01-01 03:00:00
2016-01-01 04:00:00 2016-01-01 04:00:00
...                                 ...
2021-04-18 02:00:00 2021-04-18 02:00:00
2021-04-18 03:00:00 2021-04-18 03:00:00
2021-04-18 04:00:00 2021-04-18 04:00:00
2021-04-18 05:00:00 2021-04-18 05:00:00
2021-04-18 06:00:00 2021-04-18 06:00:00

[46423 rows x 1 columns]


In [333]:
grid = {c: v.values for c,v in dict(wind_data.coords).items()}
grid

{'longitude': array([32.  , 32.25, 32.5 , 32.75, 33.  ], dtype=float32),
 'latitude': array([32.  , 31.75, 31.5 , 31.25, 31.  , 30.75, 30.5 , 30.25, 30.  ,
        29.75, 29.5 , 29.25, 29.  ], dtype=float32),
 'time': array(['2016-01-01T00:00:00.000000000', '2016-01-01T01:00:00.000000000',
        '2016-01-01T02:00:00.000000000', ...,
        '2021-04-18T04:00:00.000000000', '2021-04-18T05:00:00.000000000',
        '2021-04-18T06:00:00.000000000'], dtype='datetime64[ns]')}

In [334]:
def interpolate(dataset, fill_value=None, drop_coords=(), **kwargs):    
    
    if not set(kwargs)<=set(dataset.coords):
        raise KeyError()
        
    if not set(drop_coords)<=set(dataset.coords):
        raise KeyError()
    
    if set(drop_coords)&set(kwargs)!=set():
        raise KeyError()
        
    for dim in dataset.coords:
        if dim not in kwargs:
            kwargs[dim] = dataset[dim].values
    
    points = dict(zip(kwargs.keys(), np.broadcast_arrays(*kwargs.values())))
    points = dict(filter(lambda p: (p[0] in dataset.dims), points.items()))
    
    ndims = next(iter(points.values())).ndim
    for k,v in points.items():
        points[k] = xr.DataArray(v, dims=[str(i) for i in range(ndims)])
        
    interp_data = {}
        
    for dat in dataset.data_vars:
        
        interp = dataset[dat].interp(**points)
        
        if len(drop_coords)>0:
            for dc in drop_coords:
                interp = interp.reduce(np.nanmin, dim=dc)
                interp = interp.squeeze(dim=dc, drop=True)
        
        if fill_value!=None:
            interp = interp.fillna(fill_value)
            
        interp_data[dat] = interp.values
            
    return interp_data

In [335]:
def wind(time, lat, lon):
    
    time = np.array(time, dtype=np.datetime64)
    time, lat, lon = np.broadcast_arrays(time, lat, lon)
    
    lon = lon%360
    interp = interpolate(wind_data, time=time, latitude=lat, longitude=lon)
    
    return interp["u10"]/scpcst.knot, interp["v10"]/scpcst.knot, interp["i10fg"]/scpcst.knot

In [336]:
def beaufort_continuous(wind_speed):
    
    wind_speed = scpcst.knot*wind_speed
    return (wind_speed/0.836)**(2/3)

In [337]:
Beaufort = np.array([1,4,7,11,17,22,28,34,41,48,56,64,160]) ## https://en.wikipedia.org/wiki/Beaufort_scale

In [338]:
def beaufort_discrete(wind_speed):
    
    wind_speed = np.array(wind_speed)
    beaufort = np.full_like(wind_speed, 0)
    
    for i in range(Beaufort.size):
        if i==0:
            mask = (wind_speed<Beaufort[i])
        elif i<Beaufort.size-1:
            mask = (wind_speed>=Beaufort[i-1]) & (wind_speed<Beaufort[i])
        else:
            mask = (wind_speed>=Beaufort[i-1])
        
        beaufort[mask] = i
    
    return beaufort

In [339]:
v = np.linspace(0,100,200)
a = beaufort_continuous(v)
b = beaufort_discrete(v)
px.scatter(x=v, y=[a,b], template="plotly_dark")

# Points of interest

In [340]:
points = pd.DataFrame({"point": ["Suez"],
                       "latitude": [30.007000],
                       "longitude": [32.581667]})
points

,point,latitude,longitude
0,Suez,30.007,32.581667


In [341]:
#"Suez",Livourne","Moroni", "Santos"
#30.007000, 43.5398079,-11.7009062, -23.9715768
#32.581667, 10, 43, -46.2970715

In [342]:
px.scatter_mapbox(points, lat="latitude", lon="longitude", template="plotly_dark", zoom=10, mapbox_style="open-street-map", color="point")

In [343]:
def wind_rose(lat, lon, start_time, end_time, heading=0, speeds=None, num_directions=8, squeeze=0.1):
    
    if (lat<grid["latitude"].min() or lat>grid["latitude"].max()):
        raise ValueError()
    if (lon<grid["longitude"].min() or lon>grid["longitude"].max()):
        raise ValueError()
        
        
    start_time = np.datetime64(start_time)
    end_time = np.datetime64(end_time)
    
    if max(start_time, end_time)<grid["time"].min():
        raise ValueError()
    if min(start_time, end_time)>grid["time"].max():
        raise ValueError()
        
    times = grid["time"].copy()
    times = times[times>=start_time]
    times = times[times<end_time]
    
    
    if speeds is None:
        scale_is_beaufort = True
        speeds = Beaufort.copy()
    else:
        scale_is_beaufort = False
        speeds = np.array(speeds)
    
    speeds.sort()
    if speeds[0]==0:
        raise ValueError()
    
    
    directions = (360/num_directions) * (np.arange(num_directions)+1/2)
    
        
    u,v, gusts = wind(times, lat, lon)
    
    angle = (np.rad2deg(np.arctan2(u,v)) + 180 - heading)%360
    wind_speed = np.sqrt(u**2+v**2)
    
    
    def freq_matrix(v, a):
        
        freq = np.zeros((directions.size, speeds.size+1))
        
        for i in range(directions.size):

            if i==0:
                p = v[ (a<directions[0]) | (a>=directions[-1]) ]
            else:
                p = v[ (a>=directions[i-1]) & (a<directions[i]) ]

            for j in range(speeds.size+1):

                if j==0:
                    q = p[ p<speeds[0] ]
                elif j<speeds.size:
                    q = p[ (p>=speeds[j-1]) & (p<speeds[j]) ]
                else:
                    q = p[ p>=speeds[-1] ]
                    
                freq[i,j] = len(q)/len(v)
                
        return freq
    
    
    squeeze_angle = squeeze*360/num_directions
    angle_range = directions[0] + np.arange(360.)
    
    
#     fig = px.scatter_polar()
    fig = psp.make_subplots(rows=1, cols=1,
                            specs=[[{'type': 'polar'}]],
                            subplot_titles=["Winds"])
    
    fig.print_grid()
    #return
    
    for k, data in enumerate([wind_speed]):
    
        freq_data = freq_matrix(data, angle)
        cum_data = np.cumsum(freq_data, axis=1)
        cum_data_360 = np.zeros((angle_range.size, speeds.size+1))
        
        for i in range(directions.size):
            
            if i==0:
                mask_angle = (angle_range<directions[0]-squeeze_angle) | (angle_range>=directions[-1]+squeeze_angle)
            else:
                mask_angle = (angle_range>=directions[i-1]+squeeze_angle) & (angle_range<directions[i]-squeeze_angle)
            
            for j in range(speeds.size+1):
                
                cum_data_360[mask_angle, j] = cum_data[i,j]
                
        
        for j in range(speeds.size+1):
                
            inner_curve = (cum_data_360[:,j-1] if j>0 else np.zeros(angle_range.size))
            outer_curve = cum_data_360[:,j]
                
            color = f"hsla({360*j/(speeds.size+1)}, 100, 50, 0.8)"
            
            if scale_is_beaufort==False:
                name = f"{speeds[j-1] if j>0 else 0} kn <= W < " + (f"{speeds[j]} kn" if j>=speeds.size else "")
            else:
                name = f"BF {j}"
            
            fig.add_trace(pgo.Scatterpolar(r = inner_curve,
                                           theta = angle_range,
                                           mode = "lines",
                                           line = {"color": "rgba(0,0,0,0)"},
                                           hoverinfo = "skip",
                                           showlegend = False),
                         row=1, col=1)

            fig.add_trace(pgo.Scatterpolar(r = outer_curve,
                                           theta = angle_range,
                                           mode = "lines",
                                           fill = "tonext",
                                           fillcolor = color,
                                           line = {"color": "black", "width": 1},
                                           name = name,
                                           showlegend = (True if k==0 else False)),
                          row=1, col=1)
            
            
    fig.update_layout(template="plotly_dark")
    
    fig.update_annotations(yshift=30)

    fig.update_layout(title = {"text": f"Statistical wind data for Suez Canal at ({lat} °N, {lon} °E)", "x": 0.5, "y": 0.95, "yanchor": "bottom", "font": {"size": 20}},
                      margin = {'l':40,'t':120,'b':50,'r':20},
                      polar1 = dict(radialaxis = {"tickformat": ".0%"},
                                    radialaxis_angle = 0,
                                    angularaxis = {"rotation":90, "direction": "clockwise"}),
                      polar2 = dict(radialaxis = {"tickformat": ".0%"},
                                    radialaxis_angle = 0,
                                    angularaxis = {"rotation":90, "direction": "clockwise"}))
        
    return fig
                
                
#fig.write_html("Figures\\FC vs Speed.html", config={"scrollZoom":True, "displayModeBar": True, "toImageButtonOptions": {"format":"png", "height":800, "width":1000}})
#fig.show(config={"scrollZoom":True, "displayModeBar": True, "toImageButtonOptions": {"format":"png", "height":800, "width":1000}})            
            
    

In [344]:
def write_fig(fig, path):
    
    fig.write_html(path, config={"scrollZoom":True, "displayModeBar": True, "toImageButtonOptions": {"format":"png", "height":800, "width":1000}})

In [345]:
wind_rose(30.007000, 32.581667, datetime.datetime(2017,1,1), datetime.datetime(2017,2,1), speeds=None, num_directions=8)

This is the format of your plot grid:
[ (1,1) polar ]



In [346]:
def wind_rose1(lat, lon, start_time, end_time, heading=0, speeds=None, num_directions=8, squeeze=0.1):
    
    if (lat<grid["latitude"].min() or lat>grid["latitude"].max()):
        raise ValueError()
    if (lon<grid["longitude"].min() or lon>grid["longitude"].max()):
        raise ValueError()
        
        
    start_time = np.datetime64(start_time)
    end_time = np.datetime64(end_time)
    
    if max(start_time, end_time)<grid["time"].min():
        raise ValueError()
    if min(start_time, end_time)>grid["time"].max():
        raise ValueError()
        
    times = grid["time"].copy()
    times = times[times>=start_time]
    times = times[times<end_time]
    
    
    if speeds is None:
        scale_is_beaufort = True
        speeds = Beaufort.copy()
    else:
        scale_is_beaufort = False
        speeds = np.array(speeds)
    
    speeds.sort()
    if speeds[0]==0:
        raise ValueError()
    
    
    directions = (360/num_directions) * (np.arange(num_directions)+1/2)
    
        
    u,v, gusts = wind(times, lat, lon)
    
    angle = (np.rad2deg(np.arctan2(u,v)) + 180 - heading)%360
    wind_speed = np.sqrt(u**2+v**2)
    
    
    def freq_matrix(v, a):
        
        freq = np.zeros((directions.size, speeds.size+1))
        
        for i in range(directions.size):

            if i==0:
                p = v[ (a<directions[0]) | (a>=directions[-1]) ]
            else:
                p = v[ (a>=directions[i-1]) & (a<directions[i]) ]

            for j in range(speeds.size+1):

                if j==0:
                    q = p[ p<speeds[0] ]
                elif j<speeds.size:
                    q = p[ (p>=speeds[j-1]) & (p<speeds[j]) ]
                else:
                    q = p[ p>=speeds[-1] ]
                    
                freq[i,j] = len(q)/len(v)
                
        return freq
    
    
    squeeze_angle = squeeze*360/num_directions
    angle_range = directions[0] + np.arange(360.)
    
    
#     fig = px.scatter_polar()
    fig = psp.make_subplots(rows=1, cols=1,
                            specs=[[{'type': 'polar'}]],
                            subplot_titles=["Winds Gust"])
    
    fig.print_grid()
    #return
    
    for k, data in enumerate([gusts]):
    
        freq_data = freq_matrix(data, angle)
        cum_data = np.cumsum(freq_data, axis=1)
        cum_data_360 = np.zeros((angle_range.size, speeds.size+1))
        
        for i in range(directions.size):
            
            if i==0:
                mask_angle = (angle_range<directions[0]-squeeze_angle) | (angle_range>=directions[-1]+squeeze_angle)
            else:
                mask_angle = (angle_range>=directions[i-1]+squeeze_angle) & (angle_range<directions[i]-squeeze_angle)
            
            for j in range(speeds.size+1):
                
                cum_data_360[mask_angle, j] = cum_data[i,j]
                
        
        for j in range(speeds.size+1):
                
            inner_curve = (cum_data_360[:,j-1] if j>0 else np.zeros(angle_range.size))
            outer_curve = cum_data_360[:,j]
                
            color = f"hsla({360*j/(speeds.size+1)}, 100, 50, 0.8)"
            
            if scale_is_beaufort==False:
                name = f"{speeds[j-1] if j>0 else 0} kn <= W < " + (f"{speeds[j]} kn" if j>=speeds.size else "")
            else:
                name = f"BF {j}"
            
            fig.add_trace(pgo.Scatterpolar(r = inner_curve,
                                           theta = angle_range,
                                           mode = "lines",
                                           line = {"color": "rgba(0,0,0,0)"},
                                           hoverinfo = "skip",
                                           showlegend = False),
                         row=1, col=1)

            fig.add_trace(pgo.Scatterpolar(r = outer_curve,
                                           theta = angle_range,
                                           mode = "lines",
                                           fill = "tonext",
                                           fillcolor = color,
                                           line = {"color": "black", "width": 1},
                                           name = name,
                                           showlegend = (True if k==0 else False)),
                          row=1, col=1)
            
            
    fig.update_layout(template="plotly_dark")
    
    fig.update_annotations(yshift=30)

    fig.update_layout(title = {"text": f"Statistical wind Gust data for Suez Canal at ({lat} °N, {lon} °E)", "x": 0.5, "y": 0.95, "yanchor": "bottom", "font": {"size": 20}},
                      margin = {'l':40,'t':120,'b':50,'r':20},
                      polar1 = dict(radialaxis = {"tickformat": ".0%"},
                                    radialaxis_angle = 0,
                                    angularaxis = {"rotation":90, "direction": "clockwise"}),
                      polar2 = dict(radialaxis = {"tickformat": ".0%"},
                                    radialaxis_angle = 0,
                                    angularaxis = {"rotation":90, "direction": "clockwise"}))
        
    return fig
                
                
#fig.write_html("Figures\\FC vs Speed.html", config={"scrollZoom":True, "displayModeBar": True, "toImageButtonOptions": {"format":"png", "height":800, "width":1000}})
#fig.show(config={"scrollZoom":True, "displayModeBar": True, "toImageButtonOptions": {"format":"png", "height":800, "width":1000}})            
            
    

In [347]:
wind_rose1(30.007000, 32.581667, datetime.datetime(2017,1,1), datetime.datetime(2017,2,1), speeds=None, num_directions=8)

This is the format of your plot grid:
[ (1,1) polar ]



In [348]:
for row in points.iterrows():
    lat = row[1]["latitude"]
    lon = row[1]["longitude"]
    fig = wind_rose(lat, lon, datetime.datetime(2017,1,1), datetime.datetime(2020,12,31), speeds=None, num_directions=8)
    write_fig(fig, fr"wind_{row[1]['point']}.html")

This is the format of your plot grid:
[ (1,1) polar ]



In [349]:
for row in points.iterrows():
    lat = row[1]["latitude"]
    lon = row[1]["longitude"]
    fig = wind_rose1(lat, lon, datetime.datetime(2017,1,1), datetime.datetime(2020,12,31), speeds=None, num_directions=8)
    write_fig(fig, fr"windGust_{row[1]['point']}.html")

This is the format of your plot grid:
[ (1,1) polar ]



In [26]:
write_fig(px.scatter_mapbox(points, lat="latitude", lon="longitude", template="plotly_dark", zoom=2, mapbox_style="open-street-map", color="point"), "Key_Locations.html")